In [ ]:
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
import diff_register.im_process as imp
import skimage.io as sio
import diff_classifier.aws as aws
%matplotlib inline

In [ ]:
rfolder = 'microglia'
lfolder = 'data'
fname = 'mglia_small'
ext = 'tif'
rfile = '{}/{}.{}'.format(rfolder, fname, ext)
lfile = '{}/{}.{}'.format(lfolder, fname, ext)
file = '{}.{}'.format(fname, ext)

In [ ]:
skeleton0, branch_data_short, nbranches, short_image, props = imp.skeleton_image(lfolder, file, show=True)

In [ ]:
X = np.zeros((len(props)))
Y = np.zeros((len(props)))
perimeter = np.zeros((len(props)))
areas = np.zeros((len(props)))
eccentricity = np.zeros((len(props)))
inertia_tensor = [0]*len(props)
label = [0]*len(props)
max_intensity = np.zeros((len(props)))
mean_intensity = np.zeros((len(props)))
moments = [0]*len(props)
solidity = np.zeros((len(props)))
total_processes = np.zeros((len(props)))
avg_p_ength = np.zeros((len(props)))
main_process = np.zeros((len(props)))

counter = 0
for item in props:
    X[counter] = item.centroid[0]
    Y[counter] = item.centroid[1]
    perimeter[counter] = item.perimeter
    areas[counter] = item.area
    eccentricity[counter] = item.eccentricity
    inertia_tensor[counter] = item.inertia_tensor
    label[counter] = item.label
    #max_intensity[counter] = item.max_intensity
    #mean_intensity[counter] = item.mean_intensity
    moments[counter] = item.moments
    solidity[counter] = item.solidity
    counter = counter + 1

features = pd.DataFrame({ 'X' : X,
                          'Y' : Y,
                          'perimeter' : perimeter,
                          'area' : areas,
                          'eccentricity' : eccentricity,
                          'inertia_tensor' : inertia_tensor,
                          'label' : label,
                          #'max intensity' : max_intensity,
                          #'mean intensity' : mean_intensity,
                          'moments' : moments,
                          'solidity' : solidity,
                        })

In [ ]:
features

In [ ]:
mglia = branch_data_short['skeleton-id'].max()
nbranches = []
xs = []
ys = []

ncount = 0
for i in branch_data_short['skeleton-id'].unique():
    bcount = branch_data_short[branch_data_short['skeleton-id']==i]['skeleton-id'].count()
    bavg = np.mean(branch_data_short[branch_data_short['skeleton-id']==i]['branch-distance'])
    blong = np.max(branch_data_short[branch_data_short['skeleton-id']==i]['branch-distance'])
    xcoord = np.mean([np.mean(branch_data_short[branch_data_short['skeleton-id']==i]['img-coord-0-0']), 
                     np.mean(branch_data_short[branch_data_short['skeleton-id']==i]['img-coord-1-0'])])
    xs.append(xcoord)
    ycoord = np.mean([np.mean(branch_data_short[branch_data_short['skeleton-id']==i]['img-coord-0-1']), 
                     np.mean(branch_data_short[branch_data_short['skeleton-id']==i]['img-coord-1-1'])])
    ys.append(ycoord)
    nbranches.append(bcount)

In [ ]:
nbranches_ord = [0]*len(nbranches)
nbranches_ord = [0]*len(nbranches)
nbranches_ord = [0]*len(nbranches)

for i in range(0, len(xs)):
    #print(xs[i], ys[i])
    skel_id = i
    min_function = np.square(xs[i] - features['X'])+np.square(ys[i] - features['Y'])
    mglia_id = np.argmin(min_function)
    nbranches_ord[mglia_id] = nbranches[skel_id]
    #print(mglia_id)
    #print(np.min(min_function))

In [ ]:
for i in range(0, features.shape[0]):
    print(np.min(np.square(features['X'])))

In [ ]:
nbranches_ord

In [ ]:
branch_data_short